# PREDICTING RSA CLUSTER LABELS

This notebook uses ANN and KNN to predict RSA cluster labels. In here, we try 50 models from 10 different seeds and chose the best according to validation accuracy and loss. Moreover, we perform 10 repeat 10-fold cross validation to ensure the model works well across a range of different training data folds.

## IMPORTING NECESSARY PACKAGES

In [240]:
from fragsys_ml import *
from keras.layers import Dropout

## READING INPUT DATA

In [3]:
main_dir = "./../"
results_dir = os.path.join(main_dir, "results")
figs_dir = os.path.join(main_dir, "figs")

In [3]:
rsa_cluster_dict = load_pickle(os.path.join(results_dir, "rsa_cluster_dict.pkl"))
cluster_membership = load_pickle(os.path.join(results_dir, "rsa_cluster_membership.pkl"))
rsa_profs = load_pickle(os.path.join(results_dir, "rsa_profs.pkl"))
ud_mat = pd.read_pickle(os.path.join(results_dir, "UD_df.pkl"))

## CREATING FEATURE DATAFRAME

In [4]:
max_len = max([len(v) for v in rsa_profs.values()])
bs_vectors = []
bs_vectors_dict = {}
for bs_id, rsa_sig in rsa_profs.items():
    rsa_sig_len = len(rsa_sig)
    rsa_range_prop = [0 for i in range(10)] # now let us change to 10
    for rsa in rsa_sig:
        prop_i = int(rsa/10) # 10 RSA BINS: b1 = [0,10), b2 = [10, 20), ... b10 = [90, MAX)
        if prop_i > 9: # if greater than 100, put in 10th bin
            prop_i = 9
        #print(prop_i)
        rsa_range_prop[prop_i] += 1
    rsa_range_prop = [round(i/rsa_sig_len, 3) for i in rsa_range_prop]
    rsa_range_prop.insert(0, rsa_sig_len/max_len) # ADDING BINDING SITE SIZE RELATIVE TO MAX SITE SIZE (IN THIS CASE 40)
    bs_vectors.append(rsa_range_prop)
    bs_vectors_dict[bs_id] = rsa_range_prop
    
vector_df = pd.DataFrame(bs_vectors, index = list(rsa_profs.keys())) # obtaining RSA vectors, which are the 11-element features used for the machine learning

In [5]:
vector_df.head(3)

,0,1,2,3,4,5,6,7,8,9,10
H0Y4R8_0_BS0,0.150,0.000,0.000,0.167,0.167,0.333,0.000,0.167,0.167,0.0,0.0
O43809_0_BS0,0.600,0.375,0.250,0.083,0.167,0.042,0.042,0.000,0.042,0.0,0.0
O43809_0_BS1,0.325,0.231,0.077,0.231,0.077,0.077,0.077,0.077,0.154,0.0,0.0


In [6]:
vector_df.shape

(293, 11)

In [8]:
#vector_df.to_pickle(os.path.join(results_dir, "rsa_vectors.pkl"))

In [7]:
label_df = pd.DataFrame.from_dict(rsa_cluster_dict, orient = "index", columns = ["label"])

In [8]:
label_df = label_df - 1

In [9]:
round(label_df.label.value_counts().sort_index()/len(label_df),2)

0    0.16
1    0.43
2    0.31
3    0.10
Name: label, dtype: float64

In [10]:
label_df.head(3)

,label
H0Y4R8_0_BS0,3
O43809_0_BS0,0
O43809_0_BS1,1


In [11]:
label_df.shape

(293, 1)

## CREATE MODEL

In [12]:
X = vector_df.sort_index() # here, X is still a dataframe. Thus, we maintain binding site IDs
y = label_df.sort_index()    # here, y is still a list

#### DATA SPLIT IN MODEL (10/11) (USED FOR TRAINING) AND BLIND (USED FOR FINAL TESTING) (1/11) STRATIFYING BY CLUSTER LABEL, SO THEY ARE BALANCED

In [13]:
X_model, X_blind, y_model, y_blind = train_test_split(X, y, test_size = 0.091, random_state = 12345, shuffle = True, stratify = y) ### X_blind and y_blind will be the blind test at the end

In [23]:
#dump_pickle(X_model.index.tolist(), os.path.join(results_dir, "X_model.pkl"))

In [14]:
y_model.head(3)

,label
P47811_0_BS1,1
P0DTD1_0_BS23,1
Q96HY7_0_BS0,1


#### CLASSES ARE IMBALANCED DUE TO OUR TRAINING DATASET, BUT BOTH TRAINING AND TEST DATA HAVE THE SAME PROPORTION FOR EACH CLASS

In [15]:
print(len(X_model))
print(len(X_blind))
print(len(y_model))
print(len(y_blind))
for el in [0,1,2,3]:
    print(el, round(y_model.label.tolist().count(el)/len(y_model),2))
for el in [0,1,2,3]:
    print(el, round(y_blind.label.tolist().count(el)/len(y_blind),2))

266
27
266
27
0 0.16
1 0.43
2 0.31
3 0.1
0 0.15
1 0.44
2 0.3
3 0.11


#### CALLBACK TO GET BEST MODEL DURING TRAINING

In [16]:
callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', min_delta = 0.01, patience = 100, verbose = 1, mode = 'max', restore_best_weights = True)

#### OBTAINING CLASS WEIGHTS TO MINIMISE EFFECT OF CLASS IMBALANCE IN TRAINING DATASET

In [17]:
class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(y_model.label.tolist()),
                                                 y = np.array(y_model.label))
class_weights_dict = {i: class_weights[i] for i in np.unique(y_model)}

In [18]:
class_weights

array([1.58333333, 0.57826087, 0.80120482, 2.55769231])

#### THE WEIGHT FOR EACH CLASS IS INVERSELY PROPORTIONAL TO THEIR FREQUENCY IN THE DATA SET

In [19]:
for k, v in class_weights_dict.items():
    print(k, round(v, 2))

0 1.58
1 0.58
2 0.8
3 2.56


#### DATA SPLIT IN MODEL_TRAIN (0.8) (USED FOR MODEL TRAINING) AND MODEL_VAL (0.2) (USED FOR MODEL VALIDATION DURING TRAINING) STRATIFYING BY CLUSTER LABEL, SO THEY ARE BALANCED

In [20]:
X_model_train, X_model_val, y_model_train, y_model_val = train_test_split(X_model, y_model, test_size = 0.2, random_state = 12345, shuffle = True, stratify = y_model)

In [21]:
len(set(X_model_train.index.tolist()).intersection(set(y_model_train.index.tolist())))/len(y_model_train)

1.0

In [22]:
len(set(X_model_val.index.tolist()).intersection(set(y_model_val.index.tolist())))/len(y_model_val)

1.0

#### TRAINING AND VALIDATION DATA HAVE THE SAME PROPORTION FOR EACH CLASS

In [23]:
print(len(X_model_train))
print(len(X_model_val))
print(len(y_model_train))
print(len(y_model_val))
for el in [0,1,2,3]:
    print(el, round(y_model_train.label.tolist().count(el)/len(y_model_train),2))
for el in [0,1,2,3]:
    print(el, round(y_model_val.label.tolist().count(el)/len(y_model_val),2))

212
54
212
54
0 0.16
1 0.43
2 0.31
3 0.1
0 0.17
1 0.43
2 0.31
3 0.09


# TRAINING

In [24]:
random.seed(1)

### USING 100 DIFFERENT SEEDS

In [25]:
seeds = [random.randint(1, 100000) for i in range(100)]

In [27]:
print(len(set(seeds)))

100


In [28]:
# seeds = 
# [
#     17612, 74607, 8272, 33433, 15456, 64938, 99741, 58916, 61899, 85406,
#     49757, 27520, 12303, 63945, 3716, 51094, 56724, 79619, 99914, 277,
#     91205, 58378, 34909, 94574, 29985, 77484, 13400, 41607, 4010, 2926,
#     3336, 85138, 70965, 1207, 49966, 89979, 28391, 55328, 95139, 3807,
#     69158, 29058, 57395, 64988, 72465, 30551, 45312, 30261, 88716, 28677,
#     99739, 60242, 37983, 2817, 54550, 72936, 84187, 13108, 24368, 82491,
#     94849, 38849, 15846, 97406, 43608, 94567, 93218, 65641, 55327, 66548,
#     87859, 24884, 39764, 37246, 77016, 65453, 66229, 51558, 77202, 4526,
#     62945, 31817, 97483, 52991, 54305, 87130, 22677, 48120, 71933, 92149,
#     88407, 96760, 49114, 11334, 57536, 87001, 66641, 14147, 21457, 68281
# ]

[17612,
 74607,
 8272,
 33433,
 15456,
 64938,
 99741,
 58916,
 61899,
 85406,
 49757,
 27520,
 12303,
 63945,
 3716,
 51094,
 56724,
 79619,
 99914,
 277,
 91205,
 58378,
 34909,
 94574,
 29985,
 77484,
 13400,
 41607,
 4010,
 2926,
 3336,
 85138,
 70965,
 1207,
 49966,
 89979,
 28391,
 55328,
 95139,
 3807,
 69158,
 29058,
 57395,
 64988,
 72465,
 30551,
 45312,
 30261,
 88716,
 28677,
 99739,
 60242,
 37983,
 2817,
 54550,
 72936,
 84187,
 13108,
 24368,
 82491,
 94849,
 38849,
 15846,
 97406,
 43608,
 94567,
 93218,
 65641,
 55327,
 66548,
 87859,
 24884,
 39764,
 37246,
 77016,
 65453,
 66229,
 51558,
 77202,
 4526,
 62945,
 31817,
 97483,
 52991,
 54305,
 87130,
 22677,
 48120,
 71933,
 92149,
 88407,
 96760,
 49114,
 11334,
 57536,
 87001,
 66641,
 14147,
 21457,
 68281]

In [29]:
initializer = tf.keras.initializers.GlorotUniform(seed = 3551)

In [30]:
X_model_train.head(3)

,0,1,2,3,4,5,6,7,8,9,10
P0DTD1_2_BS0,0.550,0.136,0.227,0.045,0.182,0.091,0.000,0.136,0.045,0.091,0.045
Q6PJP8_0_BS9,0.175,0.000,0.000,0.429,0.143,0.000,0.000,0.286,0.143,0.000,0.000
Q8IU60_0_BS0,0.300,0.333,0.000,0.167,0.250,0.083,0.083,0.000,0.000,0.000,0.083


In [4]:
wd = os.path.join(main_dir, "ANN_results/params_explore")
if not os.path.isdir(wd):
    os.mkdir(wd)

In [282]:
print(len(seeds))

100


# CURRENT ML SETUP

In [283]:
param_folder = os.path.join(wd, "current")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

STARTING SEED 17612
Restoring model weights from the end of the best epoch: 36.
Epoch 136: early stopping
STARTING SEED 74607
Restoring model weights from the end of the best epoch: 86.
Epoch 186: early stopping
STARTING SEED 8272
Restoring model weights from the end of the best epoch: 31.
Epoch 131: early stopping
STARTING SEED 33433
Restoring model weights from the end of the best epoch: 42.
Epoch 142: early stopping
STARTING SEED 15456
Restoring model weights from the end of the best epoch: 98.
Epoch 198: early stopping
STARTING SEED 64938
Restoring model weights from the end of the best epoch: 46.
Epoch 146: early stopping
STARTING SEED 99741
Restoring model weights from the end of the best epoch: 37.
Epoch 137: early stopping
STARTING SEED 58916
Restoring model weights from the end of the best epoch: 71.
Epoch 171: early stopping
STARTING SEED 61899
Restoring model weights from the end of the best epoch: 39.
Epoch 139: early stopping
STARTING SEED 85406
Restoring model weights fro

# REMOVING HIDDEN LAYER

In [ ]:
param_folder = os.path.join(wd, "no_layer")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        #Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

STARTING SEED 17612
Restoring model weights from the end of the best epoch: 123.
Epoch 223: early stopping
STARTING SEED 74607
Restoring model weights from the end of the best epoch: 123.
Epoch 223: early stopping
STARTING SEED 8272
Restoring model weights from the end of the best epoch: 159.
Epoch 259: early stopping
STARTING SEED 33433
STARTING SEED 15456
STARTING SEED 64938
STARTING SEED 99741
STARTING SEED 58916
STARTING SEED 61899
Restoring model weights from the end of the best epoch: 74.
Epoch 174: early stopping
STARTING SEED 85406
STARTING SEED 49757
STARTING SEED 27520
STARTING SEED 12303
STARTING SEED 63945
Restoring model weights from the end of the best epoch: 171.
Epoch 271: early stopping
STARTING SEED 3716
STARTING SEED 51094
STARTING SEED 56724
STARTING SEED 79619
STARTING SEED 99914
Restoring model weights from the end of the best epoch: 191.
Epoch 291: early stopping
STARTING SEED 277
STARTING SEED 91205
Restoring model weights from the end of the best epoch: 160.
Ep

# 2 layers

In [ ]:
param_folder = os.path.join(wd, "2_layer")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 10, activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# 5 layers

In [ ]:
param_folder = os.path.join(wd, "5_layer")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 10, activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 10, activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 10, activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 10, activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# 10 layers

In [ ]:
param_folder = os.path.join(wd, "10_layer")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 10, activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 10, activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 10, activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 10, activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 10, activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 10, activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 10, activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 10, activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 10, activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# 1 neuron

In [ ]:
param_folder = os.path.join(wd, "1_neurons")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 1, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# 2 neurons

In [ ]:
param_folder = os.path.join(wd, "2_neurons")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 2, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# 3 neurons

In [ ]:
param_folder = os.path.join(wd, "3_neurons")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 3, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# 4 neurons

In [ ]:
param_folder = os.path.join(wd, "4_neurons")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 4, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# 5 neurons

In [ ]:
param_folder = os.path.join(wd, "5_neurons")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 5, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# 6 neurons

In [ ]:
param_folder = os.path.join(wd, "6_neurons")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 6, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# 7 neurons

In [ ]:
param_folder = os.path.join(wd, "7_neurons")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 7, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# 8 neurons

In [ ]:
param_folder = os.path.join(wd, "8_neurons")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 8, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# 9 neurons

In [ ]:
param_folder = os.path.join(wd, "9_neurons")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 9, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# 11 neurons

In [ ]:
param_folder = os.path.join(wd, "11_neurons")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 11, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# 20 neurons

In [ ]:
param_folder = os.path.join(wd, "20_neurons")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 20, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# 25 neurons

In [ ]:
param_folder = os.path.join(wd, "25_neurons")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 25, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# 50 neurons

In [ ]:
param_folder = os.path.join(wd, "50_neurons")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 50, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# 100 neurons

In [ ]:
param_folder = os.path.join(wd, "100_neurons")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 100, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# sigmoid activation function

In [ ]:
param_folder = os.path.join(wd, "sigmoid")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "sigmoid", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# ELU activation function

In [ ]:
param_folder = os.path.join(wd, "elu")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "elu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# SELU activation function

In [ ]:
param_folder = os.path.join(wd, "selu")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "selu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# EXP activation function

In [ ]:
param_folder = os.path.join(wd, "exp")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "exponential", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# tanh activation function

In [ ]:
param_folder = os.path.join(wd, "tanh")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "tanh", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Softplus activation function

In [ ]:
param_folder = os.path.join(wd, "softplus")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "softplus", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Softmax activation function

In [ ]:
param_folder = os.path.join(wd, "softmax")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "softmax", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Softsign activation function

In [ ]:
param_folder = os.path.join(wd, "softsign")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "softsign", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# He Normal initialisation

In [ ]:
param_folder = os.path.join(wd, "initialiser_he_norm")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.HeNormal(seed=seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF



# He Uniform initialisation

In [ ]:
param_folder = os.path.join(wd, "initialiser_he_uni")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.HeUniform(seed=seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF
    
    

# Random Normal initialisation

In [ ]:
param_folder = os.path.join(wd, "initialiser_rand_norm")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Random Uniform initialisation

In [ ]:
param_folder = os.path.join(wd, "initialiser_rand_uni")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Truncated Normal initialisation

In [ ]:
param_folder = os.path.join(wd, "initialiser_trunc_norm")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF
    

# Ones initialisation

In [ ]:
param_folder = os.path.join(wd, "initialiser_ones")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.Ones()
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF
    

# Zeros initialisation

In [ ]:
param_folder = os.path.join(wd, "initialiser_zeros")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.Zeros()
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF
    

# Glorot Normal initialisation

In [ ]:
param_folder = os.path.join(wd, "initialiser_glorot_normal")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotNormal(seed=seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Mean squared error loss function

In [ ]:
param_folder = os.path.join(wd, "MSE_loss_func")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "mean_squared_error", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Poisson loss function

In [ ]:
param_folder = os.path.join(wd, "poisson_loss_func")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "poisson", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# KL Divergence loss function

In [ ]:
param_folder = os.path.join(wd, "kld_loss_func")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "kl_divergence", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF
    

# Optimiser Adadelta

In [ ]:
param_folder = os.path.join(wd, "optimiser_adadelta")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=keras.optimizers.Adadelta(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Optimiser Adagrad

In [ ]:
param_folder = os.path.join(wd, "optimiser_adagrad")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=keras.optimizers.Adagrad(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Optimiser Adamax

In [ ]:
param_folder = os.path.join(wd, "optimiser_adamax")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=keras.optimizers.Adamax(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Optimiser Nadam

In [ ]:
param_folder = os.path.join(wd, "optimiser_nadam")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Optimiser Ftrl

In [ ]:
param_folder = os.path.join(wd, "optimiser_ftrl")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=keras.optimizers.Ftrl(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Optimiser SGD

In [ ]:
param_folder = os.path.join(wd, "optimiser_SGD")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Optimiser RMSprop

In [ ]:
param_folder = os.path.join(wd, "optimiser_RMSProp")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    

    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Learning rate 0.001

In [ ]:
param_folder = os.path.join(wd, "learning_rate_0.001")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.001), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Learning rate 0.005

In [ ]:
param_folder = os.path.join(wd, "learning_rate_0.005")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.005), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Learning rate 0.05

In [ ]:
param_folder = os.path.join(wd, "learning_rate_0.05")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.05), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Learning rate 0.1

In [ ]:
param_folder = os.path.join(wd, "learning_rate_0.1")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.1), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Learning rate 0.25

In [ ]:
param_folder = os.path.join(wd, "learning_rate_0.25")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.25), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Learning rate 0.5

In [ ]:
param_folder = os.path.join(wd, "learning_rate_0.5")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.5), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Learning rate 1.0

In [ ]:
param_folder = os.path.join(wd, "learning_rate_1.0")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=1.0), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Dropout layer (Rate = 0.1)

In [ ]:
param_folder = os.path.join(wd, "dropout_0.1")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dropout(0.1),
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Dropout layer (Rate = 0.25)

In [ ]:
param_folder = os.path.join(wd, "dropout_0.25")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dropout(0.25),
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Dropout layer (Rate = 0.33)

In [ ]:
param_folder = os.path.join(wd, "dropout_0.33")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dropout(0.3333),
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Dropout layer (Rate = 0.5)

In [ ]:
param_folder = os.path.join(wd, "dropout_0.5")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dropout(0.5),
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Dropout layer (Rate = 0.75)

In [ ]:
param_folder = os.path.join(wd, "dropout_0.75")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed), # THIS IS THE MODEL
        Dropout(0.75),
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# L1 regulariser

In [ ]:
param_folder = os.path.join(wd, "L1_regular")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed, kernel_regularizer='l1'), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# L2 regularisation

In [ ]:
param_folder = os.path.join(wd, "L2_regular")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed, kernel_regularizer='l2'), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# L1L2 regularisation

In [ ]:
param_folder = os.path.join(wd, "L1L2_regular")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed, kernel_regularizer='l1_l2'), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Orthogonal regularisation (rows)

In [ ]:
param_folder = os.path.join(wd, "ortho_rows_regular")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
    
    regu = tf.keras.regularizers.OrthogonalRegularizer(factor=0.01, mode="rows")
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed, kernel_regularizer=regu), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# Orthogonal regularisation (columns)

In [ ]:
param_folder = os.path.join(wd, "ortho_cols_regular")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
    
    regu = tf.keras.regularizers.OrthogonalRegularizer(factor=0.01, mode="columns")
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed, kernel_regularizer=regu), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# All L1 regularisation

In [ ]:
param_folder = os.path.join(wd, "all_L1_regular")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
    
    #regu = tf.keras.regularizers.OrthogonalRegularizer(factor=0.01, mode="columns")
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed, kernel_regularizer='l1', bias_regularizer='l1', activity_regularizer='l1'), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# All L2 regularisation

In [ ]:
param_folder = os.path.join(wd, "all_L2_regular")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
    
    #regu = tf.keras.regularizers.OrthogonalRegularizer(factor=0.01, mode="columns")
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed, kernel_regularizer='l2', bias_regularizer='l2', activity_regularizer='l2'), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# All L1L2 regularisation

In [ ]:
param_folder = os.path.join(wd, "all_L1L2_regular")

if not os.path.isdir(param_folder):
    os.mkdir(param_folder)
    
for seed in seeds:
    print("STARTING SEED {}".format(seed))
    
    mods_dir = os.path.join(param_folder, "models")
    hist_dir = os.path.join(param_folder, "hists")
    
    if not os.path.isdir(mods_dir):
        os.mkdir(mods_dir)
    if not os.path.isdir(hist_dir):
        os.mkdir(hist_dir)
    
    initializer_seed = tf.keras.initializers.GlorotUniform(seed = seed)
    
    #regu = tf.keras.regularizers.OrthogonalRegularizer(factor=0.01, mode="columns")
   
    model_i = Sequential([
        Dense(units = 10, input_shape=(11,), activation = "relu", kernel_initializer = initializer_seed, kernel_regularizer='l1_l2', bias_regularizer='l1_l2', activity_regularizer='l1_l2'), # THIS IS THE MODEL
        Dense(units = 4, activation = "softmax", kernel_initializer = initializer_seed)
    ])

    model_i.compile(optimizer=Adam(learning_rate=0.01), loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]) # COMPILE MODEL

    hist_i = model_i.fit(
        x = X_model_train, y = y_model_train, batch_size = 32,
        callbacks = [callback], validation_data = (X_model_val, y_model_val),
        epochs = 300, shuffle = True, verbose = 0, class_weight =  class_weights_dict # TRAIN MODEL
    )
    final_epoch = callback.stopped_epoch-100
    if final_epoch == -100:
        final_epoch = 299
    final_acc = round(hist_i.history["val_accuracy"][final_epoch],2)

    model_i.save(os.path.join(mods_dir, "{}_model_epoch_{}_train_acc_{}.h5".format(seed, final_epoch, round(final_acc, 2)))) # SAVING MODEL

    pd.DataFrame(hist_i.history).to_csv(os.path.join(hist_dir, "{}_hist_epoch_{}_train_acc_{}.csv".format(seed, final_epoch, round(final_acc, 2))), index = False) # SAVING TRAINING DF

# COMPARING RESULTS

In [64]:
import re

In [ ]:
diff_params = [
    "current", "no_layer", "2_layer", "5_layer", "10_layer",
    "1_neurons", "2_neurons", "3_neurons", "4_neurons", "5_neurons",
    "6_neurons", "7_neurons", "8_neurons", "9_neurons",
    "11_neurons", "20_neurons", "50_neurons", "100_neurons",
    "dropout_0.1", "dropout_0.25", "dropout_0.33", "dropout_0.5", "dropout_0.75",
    "L1_regular", "L2_regular", "L1L2_regular", "ortho_rows_regular", "ortho_cols_regular",
    "all_L1_regular", "all_L2_regular", "all_L1L2_regular",
    "sigmoid", "tanh", "elu", "selu", "exp", "softplus", "softmax", "softsign",
    "initialiser_he_norm", "initialiser_he_uni", "initialiser_rand_norm", "initialiser_rand_uni",
    "initialiser_trunc_norm", "initialiser_ones", "initialiser_zeros", "initialiser_glorot_normal",
    "MSE_loss_func", "poisson_loss_func", "kld_loss_func", "optimiser_SGD", "optimiser_RMSProp",
    "optimiser_adadelta", "optimiser_adagrad", "optimiser_adamax", "optimiser_nadam", "optimiser_ftrl",
    "learning_rate_0.001", "learning_rate_0.005", "learning_rate_0.05", "learning_rate_0.1",
    "learning_rate_0.25", "learning_rate_0.5", "learning_rate_1.0",
]

In [ ]:
p = re.compile("""(\d+)_hist_epoch_(\d+)_train_acc_(\d+\.\d+)\.csv""")

versions, seed_ids, n_epochs, accs = [[], [], [], []]

for param in diff_params:
    param_dir = os.path.join(wd, param)
    hists_dir = os.path.join(param_dir, "hists")
    hists = os.listdir(hists_dir)
    for hist in hists:
        #print(hist)
        try:
            m = p.match(hist)
            if m:
                g = m.groups()
                seed, epoch, acc = g
        except:
            print("ERROR")
        
        df = pd.read_csv(os.path.join(hists_dir, hist))
       # print(seed, epoch, acc)
        versions.append(param)
        seed_ids.append(seed)
        n_epochs.append(int(epoch))
        accs.append(df.loc[int(epoch), "val_accuracy"])
        #break
    #break



In [ ]:
master_df = pd.DataFrame(list(zip(versions, seed_ids, n_epochs, accs)),
              columns=['version','seed', 'f_epoch', 'acc'])

In [280]:
#master_df.head()

In [ ]:
plt.figure(figsize = (15, 5), dpi = 150)

# palette = {
#     'ANN': 'firebrick',
#     'KNN': 'royalblue',
#     'Random ANN': 'tomato',
#     'Random KNN': 'cornflowerblue',
#     'Random': 'gold'
# }

# alpha = 0.75

# PROPS = {
#     'boxprops':{'edgecolor':'k', 'alpha' : alpha},
#     'medianprops':{'color':'k'},
#     'whiskerprops':{'color':'k'},
#     'capprops':{'color':'k'},
    
# }

flierprops = dict(marker='o', markerfacecolor= "k", markersize=5,
                  linestyle='none', color="k", markeredgecolor='k')

# Create the boxplot with different colors
sns.boxplot(x='version', y='acc', data=master_df)

plt.ylim(-0.05, 1.1)
plt.ylabel("Validation accuracy")
plt.xticks(rotation=-90)
plt.xlabel("Model")

#plt.savefig(os.path.join(figs_dir, "ML_CV_rev.png"))
plt.show()

In [ ]:
master_df.to_pickle(os.path.join(wd, "ANN_parameter_exploration.pkl"))